In [1]:
%reload_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from docplex.cp.model import  CpoModel
import docplex.cp.utils_visu as visu

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

import matplotlib.pyplot as plt
%matplotlib inline
#Change the plot size
from pylab import rcParams
rcParams['figure.figsize'] = 15, 3

import sys
sys.path.append("..")
from utils import extract_solution

Taillard specification

On the first line are two numbers, the first is the number of jobs and the second the number of machines. Following there are two matrice the first with a line for each job containing the processor times for each operation the second with the order for visiting the machines. The numbering of the machines starts at 1.

For example an instance with only a single job on three machines where the processing time is 5 on machine 1, 6 on machine 2 and 7 on machine 3 and the order that the machines are to be visited by that job is 2,3,1. The instance would be presented as:

1	3	

6	7	5

2	3	1



In [4]:
# rows are Job Ids and columns are operations ids

processing_time = [[54,34,61,2],[9,15,89,70],[38,19,28,87],[95,34,7,29]]
processing_time = pd.DataFrame(processing_time, columns = range(1,5), index = range(1,5))

display(processing_time)

# rows are Job Ids and columns are operations ids
machines = [[3,1,4,2],[4,1,2,3],[1,2,3,4],[1,3,2,4]]
machines = pd.DataFrame(machines, columns = range(1,5), index = range(1,5))

display(machines)

,1,2,3,4
1,54,34,61,2
2,9,15,89,70
3,38,19,28,87
4,95,34,7,29


,1,2,3,4
1,3,1,4,2
2,4,1,2,3
3,1,2,3,4
4,1,3,2,4
